## Geomorphometry I: Terrain Modeling

Resources: [
GRASS GIS overview and manual](https://grass.osgeo.org/grass76/manuals/index.html),
[ libLAS tools for lidar data conversions](http://liblas.org/)


Download the Raleigh 2013 lidar data as LAS file and orthophoto:

* [LAS tile 0793_016 in NC State Plane Meters ](http://fatra.cnr.ncsu.edu/geospatial-modeling-course/data/tile_0793_016_spm.las)
* [Orthophoto geotif (mosaic at 1ft resolution)](http://fatra.cnr.ncsu.edu/geospatial-modeling-course/data/ortho_0793_016_ncspm.zip)


### Software support

The GRASS GIS installation on your computer may not support
_r.in.lidar_ and _v.in.lidar_
modules. You can try them, e.g. using

In [ ]:
!r.in.lidar --help

On Windows platform, please use the standalone version of GRASS GIS
we recommend in [Course Logistics](../logistics.html).
If the modules don't work for you, please use instead of this assignment the
[ASCII file based version](terrain_modeling_ascii.html)
of the assignment.

### Start GRASS GIS
Start GRASS - click on GRASS icon or type

In [ ]:
# using Python to initialize GRASS GIS
# This is a quick introduction into Jupyter Notebook.
# Python code can be executed like this:
a = 6
b = 7
c = a * b
print("Answer is", c)
# Python code can be mixed with command line code (Bash).
# It is enough just to prefix the command line with an exclamation mark:
!echo "Answer is $c"
# Use Shift+Enter to execute this cell. The result is below.

In [ ]:
# using Python to initialize GRASS GIS
import os
import sys
import subprocess
from IPython.display import Image

# create GRASS GIS runtime environment
gisbase = subprocess.check_output(["grass", "--config", "path"], text=True).strip()
os.environ['GISBASE'] = gisbase
sys.path.append(os.path.join(gisbase, "etc", "python"))

# do GRASS GIS imports
import grass.script as gs
import grass.script.setup as gsetup

# set GRASS GIS session data
rcfile = gsetup.init(gisbase, "/home/jovyan/grassdata", "nc_spm_08_grass7", "user1")

In [ ]:
# using Python to initialize GRASS GIS
# default font displays
os.environ['GRASS_FONT'] = 'sans'
# overwrite existing maps
os.environ['GRASS_OVERWRITE'] = '1'
gs.set_raise_on_error(True)
gs.set_capture_stderr(True)

In [ ]:
# using Python to initialize GRASS GIS
# set display modules to render into a file (named map.png by default)
os.environ['GRASS_RENDER_IMMEDIATE'] = 'cairo'
os.environ['GRASS_RENDER_FILE_READ'] = 'TRUE'
os.environ['GRASS_LEGEND_FILE'] = 'legend.txt'

In startup pannel set GRASS GIS Database Directory to path to datasets,
for example on MS Windows, `C:\Users\myname\grassdata`.
For GRASS Location select nc_spm_08_grass7 (North Carolina, State Plane, meters) and
for GRASS Mapset create a new mapset (called e.g. HW_terrain_modeling).
Click Start GRASS session.


Change working directory:
_Settings_ > _GRASS working environment_ > _Change working directory_ > select/create any directory
or type `cd` (stands for change directory) into the GUI
_Console_ and hit Enter:

Download all files (see above)
to the selected directory. Now you can use the commands from the assignment requiring the file
without the need to specify the full path to the file.



### Analyze bare earth and multiple return lidar data properties by binning
Import the points using _v.in.lidar_.
We can specify which class and which return (first, middle, last) we want to import.
We can see the classification either in metadata distributed with the lidar data
or it can be displayed with [lasinfo](http://liblas.org/) tool (in case lasinfo command is not available, skip it.):

In [ ]:
!lasinfo tile_0793_016_spm.las

Class 2 represents bare earth points.
Now we import bare earth points and first return points separately:

In [ ]:
!v.in.lidar -t input=tile_0793_016_spm.las output=elev_lid793016_be class_filter=2
!v.in.lidar -t input=tile_0793_016_spm.las output=elev_lid793016_1r return_filter=first

Note: if you have any problems with the files, review the instructions above or
use v.in.lidar GUI dialog to browse to get the path to the files.

Set the region from the imported point file with resolution of 1 meter.
Compute raster maps (with _r.in.lidar_) representing number of points per 1 m grid cell.
Compare point densities for bare earth, first return.

In [ ]:
!g.region vect=elev_lid793016_1r res=1 -p
!r.in.lidar input=tile_0793_016_spm.las output=lid_be_binn1m class_filter=2 method=n
!r.in.lidar input=tile_0793_016_spm.las output=lid_1r_binn1m return_filter=first method=n
!r.colors lid_be_binn1m,lid_1r_binn1m color=bcyr -e
!d.rast lid_be_binn1m
!d.legend lid_be_binn1m at=2,50,2,9
!r.report lid_be_binn1m unit=p,c
!r.univar lid_be_binn1m
!d.rast lid_1r_binn1m
!d.legend -d -s lid_1r_binn1m at=2,50,8,12
!r.report lid_1r_binn1m unit=p
!r.univar lid_1r_binn1m
Image(filename="map.png")

Compute a raster map representing mean elevation for each 1m cell.
It will have holes.

In [ ]:
!r.in.lidar input=tile_0793_016_spm.las output=lid_be_binmean1m class_filter=2 method=mean
!r.in.lidar input=tile_0793_016_spm.las output=lid_1r_binmean1m return_filter=first method=mean
!r.colors lid_be_binmean1m color=elevation
!r.colors lid_1r_binmean1m color=elevation
!r.info map=lid_1r_binmean1m
!d.rast lid_be_binmean1m
!d.legend lid_be_binmean1m at=2,40,2,5
!d.rast lid_1r_binmean1m
!d.legend lid_1r_binmean1m at=2,40,2,5
Image(filename="map.png")

Compute a raster map representing mean elevation for each 2m cell.
Result is almost good enough for 1st return, but there are still many holes for bare earth.

In [ ]:
!g.region res=2 -ap
!r.in.lidar input=tile_0793_016_spm.las output=lid_be_binmean2m class_filter=2 method=mean
!r.in.lidar input=tile_0793_016_spm.las output=lid_1r_binmean2m return_filter=first method=mean
!r.colors lid_be_binmean2m color=elevation
!r.colors lid_1r_binmean2m color=elevation
!d.rast lid_be_binmean2m
!d.legend lid_be_binmean2m at=2,40,2,5
!d.rast lid_1r_binmean2m
!d.legend lid_1r_binmean2m at=2,40,2,5
Image(filename="map.png")

Compute range of elevation values in each grid cell:

In [ ]:
!r.in.lidar input=tile_0793_016_spm.las output=lid_be_binrange2m class_filter=2 method=range
!r.colors lid_be_binrange2m color=bcyr -e
!r.in.lidar input=tile_0793_016_spm.las output=lid_1r_binrange2m return_filter=first method=range
!r.colors lid_1r_binrange2m color=bcyr -e
!d.rast lid_be_binrange2m
!d.legend lid_be_binrange2m at=2,40,2,5
!d.rast lid_1r_binrange2m
!d.legend lid_1r_binrange2m at=2,40,2,5
Image(filename="map.png")

Import and display orthophoto, 
switch off all layers except for orthophoto.

In [ ]:
!r.in.gdal ortho_0793_016_ncspm.tif output=ortho_2013_0793
!d.rast ortho_2013_0793
Image(filename="map.png")

Identify the features that are associated with large range values.
Display only the high values of range.
What landcover is associated with large range in multiple return data?

In [ ]:
!d.rast lid_1r_binrange2m values=10.-200.
Image(filename="map.png")

We now know how dense the data are and what is the range within cell.
If we need a 1m resolution DEM or DSM for our application
this analysis tells us that we need to interpolate it from the point cloud.
What steps would you begin with when processing
point cloud data you are not familiar with?


### Interpolate DEM and DSM

To interpolate DEM and DSM
we use default parameters except for number of points used for
segmentation and interpolation (segmax and npmin and higher tension for multiple return data).
You can set dmin=1 to make the interpolation run faster (see the manual).
Be patient, it can take a few minutes to run depending on the computer power.

In [ ]:
!g.region res=1 -ap
!v.surf.rst input=elev_lid793016_be elevation=elev_lid793016_be_1m npmin=120 segmax=25 dmin=1
!v.surf.rst input=elev_lid793016_1r elevation=elev_lid793016_1r_1m npmin=120 segmax=25 tension=100 smooth=0.5 dmin=1
!r.colors elev_lid793016_be_1m color=elevation
!r.colors elev_lid793016_1r_1m color=elevation
!d.rast elev_lid793016_be_1m
!d.rast elev_lid793016_1r_1m
Image(filename="map.png")

Hide legend and switch off all map layers except for the last 2 interpolated ones.
Use 3D view with cutting planes to compare the bare earth and terrain surface.
Make sure fine resolution is set to 1 for all surfaces.
Assign each surface constant color, add constant plane at 75m elevation for reference.
Shade the crossection using the color by bottom surface option.
If you don't remember this, see screen capture video for 3d view.


Save image for your report.

### Optional: Visualize the point cloud using plas.io

Visualizing the raw points in point clouds can be time consuming both in 2D and
3D, however several tools for doing so in an efficient way exist. Try to
use [plas.io](http://plas.io/), an online point cloud
visualization tool to visualize the LAS file from this assignment.
Explore the provided visualization tools in plas.io and create images
showing different properties of the point cloud or the visualization
techniques and how they would be useful for further analysis.

In [ ]:
# end the GRASS session
os.remove(rcfile)